In [5]:
import pandas as pd
import requests
import tempfile
import certifi

# Download the PWT data
url = 'https://www.rug.nl/ggdc/docs/pwt90.dta'
response = requests.get(url, verify=certifi.where())
response.raise_for_status()

# Save to a temporary file
with tempfile.NamedTemporaryFile(delete=False, suffix='.dta') as tmp:
    tmp.write(response.content)
    tmp_path = tmp.name

# Load the data using pandas
pwt_data = pd.read_stata(tmp_path)

# Filter for OECD countries if necessary, or use a predefined list of countries
oecd_countries = ['United States', 'Canada', 'Germany', 'France', 'United Kingdom', 'Japan', ...]  # Add other OECD countries
pwt_oecd = pwt_data[pwt_data['country'].isin(oecd_countries)]


In [6]:
# Calculate labor productivity (GDP per worker or per hour worked, depending on data availability)
pwt_oecd['labor_productivity'] = pwt_oecd['rgdpna'] / pwt_oecd['emp']  # Assuming 'emp' is employment or total labor input

# Calculate growth rate of labor productivity
pwt_oecd['labor_productivity_growth'] = pwt_oecd.groupby('country')['labor_productivity'].pct_change() * 100


/var/folders/_1/6pq414md0y1_mgwvl5dlnd2h0000gn/T/ipykernel_78117/512943561.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pwt_oecd['labor_productivity'] = pwt_oecd['rgdpna'] / pwt_oecd['emp']  # Assuming 'emp' is employment or total labor input
/var/folders/_1/6pq414md0y1_mgwvl5dlnd2h0000gn/T/ipykernel_78117/512943561.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pwt_oecd['labor_productivity_growth'] = pwt_oecd.groupby('country')['labor_productivity'].pct_change() * 100


In [8]:
# Calculate the contributions
pwt_oecd['tfp_contribution'] = pwt_oecd['rtfpna'].pct_change() * 100  # TFP growth rate
pwt_oecd['capital_contribution'] = pwt_oecd['rkna'].pct_change() * 100  # Capital deepening

# Filter data for a specific period (1960-2000)
analysis_period = pwt_oecd[(pwt_oecd['year'] >= 1960) & (pwt_oecd['year'] <= 2000)]

# Average the growth rates over the period for each country
final_table = analysis_period.groupby('country').agg({
    'labor_productivity_growth': 'mean',
    'tfp_contribution': 'mean',
    'capital_contribution': 'mean'
}).reset_index()


/var/folders/_1/6pq414md0y1_mgwvl5dlnd2h0000gn/T/ipykernel_78117/1216087066.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pwt_oecd['tfp_contribution'] = pwt_oecd['rtfpna'].pct_change() * 100  # TFP growth rate
/var/folders/_1/6pq414md0y1_mgwvl5dlnd2h0000gn/T/ipykernel_78117/1216087066.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pwt_oecd['capital_contribution'] = pwt_oecd['rkna'].pct_change() * 100  # Capital deepening


In [9]:
print(final_table)


          country  labor_productivity_growth  tfp_contribution  \
0          Canada                   1.458957          0.634422   
1          France                   3.002542          2.041511   
2         Germany                   2.858228          1.802042   
3           Japan                   4.413094          1.039314   
4  United Kingdom                   2.232333          1.212504   
5   United States                   1.808904          0.899491   

   capital_contribution  
0              4.181617  
1              4.331626  
2              3.964332  
3             11.373339  
4              2.727078  
5              3.233591  
